In [1]:
from src.config import get_parser


config = get_parser().parse_args(args=[])

In [ ]:
import torch
import torch.nn as nn

from src.data.provider import Provider
from src.models.stylist import Stylist
from src.models.generator import Generator
from src.models.discriminator import Discriminator
from src.render.mesh_renderer import MeshPointsRenderer

class Trainer:
    
    def __init__(self, config):
        self.self.G_noise_amp =  config.G_noise_amp
        self.z_size = config.latent_size
        
        self.device = torch.device("cuda")
        
        self.S = Stylist(config).to(self.device)
        self.G = Generator(config).to(self.device)
        self.D = Discriminator(config).to(self.device)
        self.R = MeshPointsRenderer(config).to(self.device)
        
        # Optimizers
        self.optim_G = torch.optim.Adam(self.G.params())
        self.optim_S = torch.optim.Adam(self.S.params())
        self.optim_D = torch.optim.Adam(self.D.params())
        # Loss functions
        self.adversarial_loss = nn.BCELoss()
        self.reconstruction_loss = nn.L1Loss()
        
    def step(self):
        pass
        
    
    def adversarial_step(self):
        batch, mean_std = self.provider.adversarial(self.device)
        images, points = batch['large'], batch['points'] # ?????
        real_label = batch['real_label']
        fake_label = batch['fake_label']
        
        # (1) Update Discriminator
        ## Train with all-real batch
        self.optim_D.zero_grad()
        output = self.D(images)
        errD_real =  self.adversarial_loss(output, real_label)
        errD_real.backward()
        D_x = output.mean().item() 
        
        ## Train with all-fake batch
        style = self.S(images)
        pts =  self.G(pts, style)
        g_images = self.R(pts, mean_std=mean_std)        
        output = self.D(g_images)
        errD_fake = self.adversarial_loss(output, label)    
        errD_fake.backward()
        
        self.optim_D.step()
        
        # (2) Update Generator and Stylist
        self.optim_S.zero_grad()
        self.optim_G.zero_grad()
        
        output = discriminator(g_images)    
        errG = self.adversarial_loss(output, real_label)
        errG.backward()
        
        self.optim_S.step()
        self.optim_G.step()
        
    def reconstruction_step(self):
        batch = self.provider.reconstruction(self.device)
        pts_coarse = batch['points_coarse']
        pts_fine = batch['points']
        vs = pts_fine.size(0)
        
        pts_noise = pts_coarse + torch.randn_like(pts_coarse) * self.G_noise_amp
        style = torch.zeros(bs, z_size, device=self.device)
        
        self.optim_G.zero_grad()
        
        vertices = self.G(pts_noise, style)
        errG_rec = self.reconstruction_loss(vertices, pts_fine)
        
        errG_rec.backward()
        
        self.optim_G.step()
        
        
        
        
        
        
        
        